In [1]:
from fractal_zero.trainers.muzero_discriminator import FractalMuZeroDiscriminatorTrainer, FMZGModel
from fractal_zero.models.joint_model import JointModel

from fractal_zero.data.expert_dataset import ExpertDatasetGenerator
from fractal_zero.vectorized_environment import load_environment


import torch

In [2]:
env = load_environment("CartPole-v0")

/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
# get_expert_action = lambda x: env.action_space.sample()  # random policy

_expert_policy_model = torch.load("models/best_cartpole_policy.pth")
def get_expert_action(x):
    embedded_actions = _expert_policy_model.forward(x)
    return _expert_policy_model.parse_actions(embedded_actions)

# play with the expert for a bit
# obs = env.reset()
# total_reward = 0
# for _ in range(200):
#     action = get_expert_action(obs)
#     obs, reward, done, info = env.step(action)
#     total_reward += reward
#     env.render()
#     if done:
#         break

# env.close()
# print(f"total_reward={total_reward}")

In [4]:
# rep_model = torch.nn.Sequential(
#     torch.nn.Linear(4, 16),
#     torch.nn.ReLU(),
#     torch.nn.Linear(16, 16),
#     torch.nn.ReLU(),
#     torch.nn.Linear(16, 4),
#     torch.nn.ReLU(),
# )
rep_model = torch.nn.Identity()

dyn_model = torch.nn.Sequential(
    torch.nn.Linear(5, 16),  # obs space + action embedding = 4 + 1 = 5
    torch.nn.ReLU(),
    torch.nn.Sigmoid(),
    torch.nn.Linear(16, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 4),
    torch.nn.Sigmoid(),  # give FMC a bit of an advantage (discrim tries to push embedding to higher magnitudes)
)

# the disc model receives the output embedding from the representation model
disc_model = torch.nn.Sequential(
    torch.nn.Linear(5, 1),
    torch.nn.Sigmoid(),  # between 0 and 1 confidence values.
)

action_vec = lambda x: torch.tensor(x).flatten().int().item()

model = FMZGModel(
    rep_model, 
    dyn_model, 
    disc_model, 
    num_walkers=256,
    action_vectorizer=action_vec,
)

expert_dataset = ExpertDatasetGenerator(
    get_expert_action,
    env,
    action_vectorizer=action_vec,
)

discriminator_optimizer = torch.optim.Adam([
    *rep_model.parameters(), 
    *dyn_model.parameters(),
    *disc_model.parameters(),
], lr=0.01)

trainer = FractalMuZeroDiscriminatorTrainer(env, model, expert_dataset, discriminator_optimizer)

In [5]:
# trainer._get_agent_trajectory(200, render=True)
# env.close()

In [6]:
max_steps = 16
for _ in range(100):
    print(trainer.train_step(max_steps), trainer.fmc.rewards.mean(), trainer.fmc.distances.mean())

/home/nollyd/Code/fractal-zero/fractal_zero/search/fmc.py:221: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  random_actions = torch.tensor(random_parsed_actions, device=self.device)


0.5787310346827896 tensor(0.3107) tensor(0.0029)
0.5705168173646845 tensor(0.3206) tensor(0.0029)
0.5640122908394857 tensor(0.3322) tensor(0.0027)
0.5579019921135092 tensor(0.3434) tensor(0.0030)
0.5491842776370115 tensor(0.3550) tensor(0.0032)
0.5423199554059845 tensor(0.3679) tensor(0.0029)
0.538463450849479 tensor(0.3799) tensor(0.0029)
0.5332719351148579 tensor(0.3912) tensor(0.0035)
0.5245437859892336 tensor(0.4030) tensor(0.0033)
0.5215429485328751 tensor(0.4144) tensor(0.0033)
0.51600447174708 tensor(0.4260) tensor(0.0036)
0.5136245416016938 tensor(0.4358) tensor(0.0034)
0.5089071846103976 tensor(0.4461) tensor(0.0030)
0.5060622042462704 tensor(0.4564) tensor(0.0032)
0.5023618517332327 tensor(0.4654) tensor(0.0028)
0.5000293621657574 tensor(0.4737) tensor(0.0023)
0.49921615197418656 tensor(0.4813) tensor(0.0018)
0.49930551517525673 tensor(0.4888) tensor(0.0014)
0.49836165831474877 tensor(0.4955) tensor(0.0013)
0.4978620525392512 tensor(0.5023) tensor(0.0011)
0.49893274239829766 

In [7]:
trainer.fmc.distances.mean()

tensor(0.0003)

In [8]:
trainer._get_agent_trajectory(max_steps, render=True)

/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


(tensor([[ 9.1259e-03,  1.8283e-02,  7.1712e-03, -4.0627e-02],
         [ 9.4915e-03,  2.1330e-01,  6.3587e-03, -3.3104e-01],
         [ 1.3758e-02,  4.0833e-01, -2.6211e-04, -6.2171e-01],
         [ 2.1924e-02,  6.0346e-01, -1.2696e-02, -9.1448e-01],
         [ 3.3993e-02,  7.9875e-01, -3.0986e-02, -1.2111e+00],
         [ 4.9968e-02,  9.9426e-01, -5.5208e-02, -1.5134e+00],
         [ 6.9853e-02,  1.1900e+00, -8.5475e-02, -1.8227e+00],
         [ 9.3654e-02,  1.3860e+00, -1.2193e-01, -2.1407e+00],
         [ 1.2137e-01,  1.5821e+00, -1.6474e-01, -2.4684e+00]],
        dtype=torch.float64),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1]))

In [9]:
env.close()